In [2]:
#IMPORTING scikit packages
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.linear_model import LinearRegression

import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
from statsmodels.sandbox.regression.predstd import wls_prediction_std

In [3]:
#IMPORTING SETUP PACKAGES
import pickle
import os.path
import os
import pandas as pd
import numpy as np

import psycopg2
import timeit
from datetime import datetime, timedelta, date
import datetime as dt
from calendar import monthrange
import random
import io
import boto3
import pyarrow as pa
import pyarrow.parquet as pq

from s3fs.core import S3FileSystem

import s3fs


%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option('display.max_rows', 5000)

In [4]:
df = pd.read_pickle('/Users/gabriele.sabato/PycharmProjects/raw_data/DataFrames/Top_seller_YTD_group_by_item_visit.pickle')

df

,min_date,max_date,item_code,item_price,delivery_weeks,avg_sales,avg_unique_views,conv_rate,bin,item_parent_item_code,item_main_category,item_sub_category_1,item_sub_category_2,log_price,log_delivery_weeks,log_sales,log_avg_unique_views,log_conv_rate
0,2020-01-01,2020-01-01,000000001000016021,119.99,5,23.000000,639.000000,0.035994,1,000000008000001083,ESSZIMMERSTÜHLE&BÄNKE,ESSZIMMERSTÜHLE,KLASSISCH,4.787409,1.609458,3.135499,6.459905,-3.321636
1,2020-01-01,2020-01-05,000000001000016133,119.99,1,10.600000,459.600000,0.023064,5,000000008000001083,ESSZIMMERSTÜHLE&BÄNKE,ESSZIMMERSTÜHLE,KLASSISCH,4.787409,0.000100,2.360863,6.130357,-3.765176
2,2020-01-01,2020-03-17,000000001000024281,59.99,1,6.857143,88.636364,0.077363,77,000000008000002127,MATRATZEN&LATTENROSTE,MATRATZEN,KALTSCHAUMMATRATZEN,4.094180,0.000100,1.925305,4.484543,-2.557960
3,2020-01-01,2020-01-05,000000001000015959,129.99,0,0.400000,11.400000,0.035088,5,000000008000001069,ESSZIMMERSTÜHLE&BÄNKE,ESSZIMMERSTÜHLE,KLASSISCH,4.867458,-9.210340,-0.916041,2.433622,-3.347058
4,2020-01-01,2020-01-01,000000001000015958,129.99,1,11.000000,728.000000,0.015110,1,000000008000001069,ESSZIMMERSTÜHLE&BÄNKE,ESSZIMMERSTÜHLE,KLASSISCH,4.867458,0.000100,2.397904,6.590301,-4.185809
5,2020-01-01,2020-02-10,000000001000022708,99.99,1,3.121951,110.682927,0.028206,41,000000008000017295,MATRATZEN&LATTENROSTE,LATTENROSTE,FEDERHOLZ,4.605071,0.000100,1.138490,4.706671,-3.564672
6,2020-01-01,2020-01-02,000000001000015944,64.99,2,8.500000,264.000000,0.032197,2,000000008000012450,SCHLAFZIMMERMÖBEL,NACHTKOMMODEN,NACHTKOMMODEN,4.174235,0.693197,2.140078,5.575949,-3.432782
7,2020-01-01,2020-04-02,000000001000022573,99.99,2,9.881720,77.688172,0.127197,93,000000008000001979,MATRATZEN&LATTENROSTE,LATTENROSTE,FEDERHOLZ,4.605071,0.693197,2.290697,4.352704,-2.061231
8,2020-01-01,2020-01-02,000000001000016019,119.99,5,6.500000,265.500000,0.024482,2,000000008000001083,ESSZIMMERSTÜHLE&BÄNKE,ESSZIMMERSTÜHLE,KLASSISCH,4.787409,1.609458,1.871818,5.581615,-3.705736
9,2020-01-01,2020-01-02,000000001000016020,119.99,2,7.500000,315.000000,0.023810,2,000000008000001083,ESSZIMMERSTÜHLE&BÄNKE,ESSZIMMERSTÜHLE,KLASSISCH,4.787409,0.693197,2.014916,5.752573,-3.733478


In [5]:
print(df.dtypes)

min_date                  object
max_date                  object
item_code                 object
item_price               float64
delivery_weeks             int64
avg_sales                float64
avg_unique_views         float64
conv_rate                float64
bin                        int32
item_parent_item_code     object
item_main_category        object
item_sub_category_1       object
item_sub_category_2       object
log_price                float64
log_delivery_weeks       float64
log_sales                float64
log_avg_unique_views     float64
log_conv_rate            float64
dtype: object


In [6]:
wls_model_with_fe = smf.wls("log_sales ~ log_price + log_delivery_weeks + log_avg_unique_views + C(item_code)", df, weights=df['bin']).fit()
print(wls_model_with_fe.summary())


                            WLS Regression Results                            
Dep. Variable:              log_sales   R-squared:                       0.647
Model:                            WLS   Adj. R-squared:                  0.624
Method:                 Least Squares   F-statistic:                     28.56
Date:                Thu, 10 Dec 2020   Prob (F-statistic):           4.30e-36
Time:                        16:19:35   Log-Likelihood:                -205.84
No. Observations:                 200   AIC:                             437.7
Df Residuals:                     187   BIC:                             480.6
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Inte

In [7]:
wls_model_with_fe_all_in = smf.wls("log_sales ~ log_price + log_delivery_weeks + log_avg_unique_views + log_conv_rate +  C(item_code)", df, weights=df['bin']).fit()
print(wls_model_with_fe_all_in.summary())



                            WLS Regression Results                            
Dep. Variable:              log_sales   R-squared:                       0.982
Model:                            WLS   Adj. R-squared:                  0.981
Method:                 Least Squares   F-statistic:                     781.5
Date:                Thu, 10 Dec 2020   Prob (F-statistic):          1.26e-154
Time:                        16:19:35   Log-Likelihood:                 91.894
No. Observations:                 200   AIC:                            -155.8
Df Residuals:                     186   BIC:                            -109.6
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Inte

In [13]:
#WLS CV rate vs the rest

wls_cr_with_fe = smf.wls("log_conv_rate ~ log_price + log_delivery_weeks + C(item_code)", df, weights=df['bin']).fit()
print(wls_cr_with_fe.summary())


                            WLS Regression Results                            
Dep. Variable:          log_conv_rate   R-squared:                       0.728
Model:                            WLS   Adj. R-squared:                  0.712
Method:                 Least Squares   F-statistic:                     45.70
Date:                Thu, 10 Dec 2020   Prob (F-statistic):           3.04e-47
Time:                        16:26:59   Log-Likelihood:                -163.24
No. Observations:                 200   AIC:                             350.5
Df Residuals:                     188   BIC:                             390.1
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Inte

In [9]:
wls_cr_with_fe = smf.wls("conv_rate ~ log_price + log_delivery_weeks + C(item_code)", df, weights=df['bin']).fit()
print(wls_cr_with_fe.summary())



                            WLS Regression Results                            
Dep. Variable:              conv_rate   R-squared:                       0.866
Model:                            WLS   Adj. R-squared:                  0.858
Method:                 Least Squares   F-statistic:                     110.4
Date:                Thu, 10 Dec 2020   Prob (F-statistic):           8.19e-76
Time:                        16:19:36   Log-Likelihood:                 475.52
No. Observations:                 200   AIC:                            -927.0
Df Residuals:                     188   BIC:                            -887.5
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Inte

In [12]:

wls_cr_del_w_with_fe = smf.wls("log_conv_rate ~ log_price + delivery_weeks + C(item_code)", df, weights=df['bin']).fit()
print(wls_cr_del_w_with_fe.summary())

                            WLS Regression Results                            
Dep. Variable:          log_conv_rate   R-squared:                       0.761
Model:                            WLS   Adj. R-squared:                  0.747
Method:                 Least Squares   F-statistic:                     54.37
Date:                Thu, 10 Dec 2020   Prob (F-statistic):           1.94e-52
Time:                        16:26:47   Log-Likelihood:                -150.31
No. Observations:                 200   AIC:                             324.6
Df Residuals:                     188   BIC:                             364.2
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Inte